# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

F. Walter  ->  F. Walter  |  ['F. Walter']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
Arxiv has 60 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2402.05165


extracting tarball to tmp_2402.05165...

 done.


E. Bañados  ->  E. Bañados  |  ['E. Bañados']
F. Walter  ->  F. Walter  |  ['F. Walter']


Found 185 bibliographic references in tmp_2402.05165/Kaasinen_CO21_QSOs.bbl.
Retrieving document from  https://arxiv.org/e-print/2402.05365


extracting tarball to tmp_2402.05365... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.05165-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.05165) | **The cold molecular gas in z>~6 quasar host galaxies**  |
|| M. Kaasinen, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2024-02-09*|
|*Comments*| *accepted for publication in A&A*|
|**Abstract**| Probing the molecular gas reservoirs of z>~6 quasar (QSO) host galaxies is fundamental to understanding the coevolution of star formation and black hole growth in these extreme systems. Yet, there is still an inhomogeneous coverage of molecular gas tracers. To measure the average excitation and mass of the molecular gas reservoirs in the brightest z>6.5 QSO hosts, we combined new observations of CO(2-1) emission with existing observations of CO(6-5), CO(7-6), [C I], [C II], and dust-continuum emission. We reduced and analysed the VLA observations of CO(2-1) in three z=6.5-6.9 QSO hosts--the highest redshift observations of CO(2-1) to date. By combining these with the nine z=5.7-6.4 QSO hosts for which CO(2-1) has already been observed, we studied the spread in molecular gas masses and CO excitation. Two of our three QSOs, were undetected in CO(2-1), implying more highly excited CO than in the z=6.4 QSO J1148+5251. We detected CO(2-1) at $5.1\sigma$ for our highest-redshift target, J2348-3054, yielding a molecular gas mass of $(1.2\pm0.2)\times 10^{10}\, \mathrm{M}_\odot$. This molecular gas mass is equivalent to the lower limit on the dynamical mass measured from resolved [C II] observations, implying little mass in stars or neutral gas within the [C II]-emitting region. On average, these QSO hosts have far higher CO(6-5)-, CO(7-6)-, and [C II] vs CO(2-1) line ratios than local AGN hosts; with a mean CO(6-5)-to-CO(1-0) line luminosity ratio of $r_{6,1}=0.9\pm0.2$. Our new CO(2-1) observations show that even at 780 Myr after the Big Bang, QSO host galaxies can already have molecular gas masses of $10^{10}$ M$_\odot$, consistent with a picture in which these z>6 QSOs reside in massive starbursts that are coevolving with the supermassive black holes. Our results imply the presence of extremely dense and warm molecular gas reservoirs illuminated by strong interstellar radiation fields. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.05365-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.05365) | **Different dependence of narrow H$α$ line luminosity on optical  continuum luminosity between starforming galaxies and Type-2 AGN: globally  negative AGN feedback in local Type-2 AGN?**  |
|| <mark>X. Zhang</mark> |
|*Appeared on*| *2024-02-09*|
|*Comments*| *16 pages, 14 figures, Accepted to be published in ApJ*|
|**Abstract**| In this manuscript, clues are provided to support globally negative AGN feedback on star formation in the host galaxies of the local low-redshift SDSS Type-2 AGN, based on the different dependence of narrow H$\alpha$ line luminosity $L_{H\alpha}$ on optical continuum luminosity $\lambda L_{cont}$ between the starforming galaxies and the Type-2 AGN. Through the measured $L_{H\alpha}$ and $\lambda L_{cont}$ in SDSS starforming galaxies, there is a strong linear correlation between $\lambda L_{cont}$ and $L_{H\alpha}$, accepted as a standard correlation without effects of AGN activity. Meanwhile, considering apparent contributions of AGN activity to narrow H$\alpha$ line emissions in the Type-2 AGN, the correlation between $\lambda L_{cont}$ and $L_{H\alpha}$ in the SDSS Type-2 AGN leads to statistically lower $L_{H\alpha}$ in the Type-2 AGN than in the starforming galaxies, with significance level higher than 5$\sigma$, even after considering necessary effects (including effects of host galaxy properties), leading to accepted conclusion on the globally negative AGN feedback in the local Type-2 AGN. Meanwhile, properties of Dn(4000) and H$\delta_A$ can provide indirect clues to support the globally negative AGN feedback in the local Type-2 AGN, due to older stellar ages in the Type-2 AGN. Moreover, it is interesting to expect more than 50\% narrow H$\alpha$ emissions globally suppressed in the host galaxies of the Type-2 AGN relative to the starforming galaxies. The results not only support globally negative AGN feedback in the local Type-2 AGN, but also show further clues on the quantification of suppressions of star formation by the globally negative AGN feedback. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2402.05165.md
    + _build/html/tmp_2402.05165/./Mmol_comp_ULIRGalphaCO_2.png
    + _build/html/tmp_2402.05165/./P036+03_284kms_3channel_map.png
    + _build/html/tmp_2402.05165/./CO21_spectrum_P036+03_2.png
    + _build/html/tmp_2402.05165/./J0305-3150_321kms_3channel_map.png
    + _build/html/tmp_2402.05165/./CO21_spectrum_J0305-3150_2.png
    + _build/html/tmp_2402.05165/./J2348-3054_548kms_3channel_map.png
    + _build/html/tmp_2402.05165/./CO21_spectrum_J2348-3054_2.png
    + _build/html/tmp_2402.05165/./mass_comp_ULIRGalphaCO.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ci}{[\ion{C}{I}] (2--1)}$
$\newcommand{\cii}{[\ion{C}{II}] 158 \textmu m}$
$\newcommand{\kms}{\ensuremath{\rm{km}  \rm{s}^{-1}}}$
$\newcommand{\peryr}{\ensuremath{\rm{yr}^{-1}}}$
$\newcommand{\Msun}{\ensuremath{\rm{M}_{\odot}}}$
$\newcommand{\Mstar}{\ensuremath{\rm{M}_{*}}}$
$\newcommand{\Mmol}{\ensuremath{\rm{M}_{\rm{mol}}}}$
$\newcommand{\aco}{\alpha_\mathrm{CO}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1}$
$\newcommand{\arraystretch}{1}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\arraystretch}{1}$</div>



<div id="title">

# The cold molecular gas in $z\gtrsim 6$ quasar host galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2402.05165-b31b1b.svg)](https://arxiv.org/abs/2402.05165)<mark>Appeared on: 2024-02-09</mark> -  _accepted for publication in A&A_

</div>
<div id="authors">

M. Kaasinen, et al. -- incl., <mark>E. Bañados</mark>, <mark>F. Walter</mark>

</div>
<div id="abstract">

**Abstract:** Probing the molecular gas reservoirs of $z\gtrsim6$ quasar (QSO) host galaxies is fundamental to understanding the coevolution of star formation and black hole growth in these extreme systems. Yet, there is still an inhomogeneous coverage of molecular gas tracers for $z\gtrsim6$ QSO hosts. To measure the average excitation and mass of the molecular gas reservoirs in the brightest $z>6.5$ QSO hosts, we combined new observations of CO(2--1) emission with existing observations of CO(6--5), CO(7--6), $\ci$ , $\cii$ , and dust-continuum emission. We reduced and analysed observations of CO(2--1), taken with the Karl G. Jansky Very Large Array, in three $z=6.5-6.9$ QSO hosts---the highest redshift observations of CO(2--1) to date. By combining these with the nine $z=5.7-6.4$ QSO hosts for which CO(2--1) emission has already been observed, we studied the spread in molecular gas masses and CO excitation of $z\gtrsim6$ QSOs. Two of our three QSOs, P036+03 and J0305--3150, were not detected in CO(2--1), implying more highly excited CO than in the well-studied $z=6.4$ QSO J1148+5251. However, we detected CO(2--1) emission at $5.1\sigma$ for our highest-redshift target, J2348--3054, yielding a molecular gas mass of $(1.2\pm0.2)\times 10^{10}  \mathrm{M}_\odot$ , assuming $\alpha_\mathrm{CO} = 0.8$ (K km s $^{-1}$ pc $^2$ ) $^{-1}$ and $r_\mathrm{2,1}=1$ . This molecular gas mass is equivalent to the lower limit on the dynamical mass measured previously from resolved $\cii$ observations, implying that there is little mass in stars or neutral gas within the [ $\ion{C}{II}$ ] -emitting region and that a low CO-to-$H_2$ conversion factor is applicable. On average, these $z\gtrsim6$ QSO hosts have far higher CO(6--5)-, CO(7--6)-, and $\cii$ versus CO(2--1) line ratios than the local gas-rich and IR-luminous galaxies that host active galactic nuclei, but with a large range of values, implying some variation in their interstellar medium conditions. We derived a mean CO(6--5)-to-CO(1--0) line luminosity ratio of $r_\mathrm{6,1}=0.9\pm0.2$ . Our new CO(2--1) observations show that even at 780 Myr after the Big Bang, QSO host galaxies can already have molecular gas masses of $10^{10}$ M $_\odot$ , consistent with a picture in which these $z\gtrsim6$ QSOs reside in massive starbursts that are coevolving with the accreting supermassive black holes. Their high gas versus dynamical masses and extremely high line excitation imply the presence of extremely dense and warm molecular gas reservoirs illuminated by strong interstellar radiation fields.

</div>

<div id="div_fig1">

<img src="tmp_2402.05165/./Mmol_comp_ULIRGalphaCO_2.png" alt="Fig1" width="100%"/>

**Figure 1. -** Comparison of molecular gas masses inferred from different sets of observations for our three QSO hosts (top three) and the literature sample of $z\gtrsim 6$ QSO hosts (bottom nine). For the CO-based values, we have assumed a ULIRG CO-to-$H_2$ conversion factor of $\alpha_\mathrm{CO}=0.8$ M$_\odot$(K km s$^{-1}$ pc$^2$)$^{-1}$.  We show two masses for CO(2--1), one derived without any CMB correction being applied (filled blue circles) and one derived by applying a correction assuming the high-density ($n_\mathrm{H_2}=10^{4.2}$ cm$^{-3}$) and high-temperature ($T_\mathrm{kin}=40$ K), non-LTE scenario presented in $\ci$te{2013ApJ...766...13D}(blue outlined circles). We also show molecular gas masses derived from $\ci$ emission (green squares) assuming the [$\ion${C}{i}] is optically thin, in LTE and has the same abundance as the sample of lensed starbursts studied in $\ci$te{2021ApJ...908...95H}, namely [$\ion${C}{I}]/$H_2 = (6.82\pm3.04)\times10^{-5}$. We also compare to molecular gas masses derived from the dust masses fit here, assuming the local gas-to-dust ratio of 100 (red triangles), as well as molecular gas masses derived from the $\ci$i line luminosities, assuming the mean [$\ion${C}{II}]-to-$H_2$ conversion factor ($\alpha_\mathrm{[CII]} = 31 $\Msun$/L_{\odot}$) derived by $\ci$te{2018MNRAS.481.1976Z}.  (*fig:Mmol_comp*)

</div>
<div id="div_fig2">

<img src="tmp_2402.05165/./P036+03_284kms_3channel_map.png" alt="Fig3.1" width="16%"/><img src="tmp_2402.05165/./CO21_spectrum_P036+03_2.png" alt="Fig3.2" width="16%"/><img src="tmp_2402.05165/./J0305-3150_321kms_3channel_map.png" alt="Fig3.3" width="16%"/><img src="tmp_2402.05165/./CO21_spectrum_J0305-3150_2.png" alt="Fig3.4" width="16%"/><img src="tmp_2402.05165/./J2348-3054_548kms_3channel_map.png" alt="Fig3.5" width="16%"/><img src="tmp_2402.05165/./CO21_spectrum_J2348-3054_2.png" alt="Fig3.6" width="16%"/>

**Figure 3. -** CO(2--1) observations of P036+03, J0305--3150, and J2348--3054 (from top to bottom, respectively). Left: Cleaned channel maps with the middle panel depicting the moment-0 map centred on the $\ci$i redshift and integrated over $1.2\times$\ac{fwhm} of the $\ci$i line, which should encompass 84\% of the CO(2--1) emission assuming there is no spatial or velocity offset. Right: Observed CO(2--1) spectra extracted from the pixel at the peak position of the moment 0 maps (blue line) and $1\sigma$ rms per channel (orange line). The spectra are shown for channels of 100 km s$^{-1}$ velocity width, the shaded region indicating the $1.2$ FWHM region used to create the channel maps to the left. P036+03 and J0305--3150 are undetected (with a significant number of negative and positive peaks in the data cube exceeding the significance of the peaks in emission near the expected source centres), so we place $4\sigma$ upper limits on their CO(2--1) emission. J2348--3054 is detected at $5.1\sigma$(central velocity-integrated map in bottom row).
				 (*fig:channelmaps*)

</div>
<div id="div_fig3">

<img src="tmp_2402.05165/./mass_comp_ULIRGalphaCO.png" alt="Fig2" width="100%"/>

**Figure 2. -** Dynamical masses (black squares), dust masses (red triangles), and black hole masses (green stars) compared to the CO(2--1)-derived molecular gas masses assuming a ULIRG $\alpha_\mathrm{CO}$, with a CMB correction (light shaded symbols) and without a CMB correction (dark shaded symbols). For the CO(2--1)-derived values, we assume that the low-$J$ CO emission is thermalised ($L_\mathrm{CO(1-0)}^\prime = L_\mathrm{CO(2-1)}^\prime$). Most molecular gas masses are high, spanning $0.1-1\times$ the dynamical mass.  (*fig:mass_comp*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2402.05165"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

368  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

2  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
